In [18]:
import pandas as pd
from collections import Counter
from PIL import Image,ImageDraw,ImageFont
from scipy.stats import spearmanr
import os
import numpy as np

In [19]:
limefeat_alphasort = pd.read_csv("limefeat_alphasort.csv")
officer_table = pd.read_csv("officer_table.csv")
df = pd.read_csv("lime-robustness-all-score-outcome.csv")

In [20]:
df.sort_values(by=['unit_score','officer_id','nft'],ascending=[False,True,True],inplace=True)
df.reset_index(drop=True,inplace=True)

In [21]:
officer_table.sort_values(by="unit_score",ascending=False,inplace=True)
officer_table.reset_index(drop=True,inplace=True)

In [22]:
def relister(liststring):
    tmp = liststring.lstrip("[").rstrip("]")
    tmp = tmp.split(",")
    tmp = [item.translate(None," '") for item in tmp]
    return tmp

In [23]:
def setstring2list(setstring):
    tmp = setstring.lstrip("{").rstrip("}")
    tmp = tmp.split(",")
    tmp = [item.translate(None," '") for item in tmp]
    return tmp

In [24]:
officer_table['feats'] = officer_table.feats.apply(relister)
df['unq'] = df.unq.apply(setstring2list)

In [25]:
X = pd.read_csv("explained_officers.csv")
del X['outcome']
del X['as_of_date']
tmp = officer_table[['officer_id','unit_score']]
tmp.set_index('officer_id',inplace=True)
X = X.join(tmp,on='officer_id')
X.sort_values(by='unit_score',ascending=False,inplace=True)
X.reset_index(drop=True,inplace=True)
del X['officer_id']
del X['unit_score']

In [26]:
feature_type_sets = []
for i in officer_table.index:
    type_list = []
    features = officer_table.feats.loc[i]
    for feature in features:
        tmp = limefeat_alphasort[limefeat_alphasort.feature==feature]
        record_type = tmp.record_type.iloc[0]
        type_list.append(record_type)
    feature_type_sets.append(set(type_list))
officer_table['feature_type_sets'] = feature_type_sets
officer_table['type_flag'] = [len(item) for item in officer_table.feature_type_sets]

#### Feature categories

In [27]:
cathue = {
    'ts':'0',
    'fi':'32',
    'ir':'60',
    'shifts':'115',
    'dispatch':'180',
    'arrests':'240',
    'ic':'270',
    'demarrests':'300',
    'ocnd':'200'
    }

catsat = {
    'ts':'100%',
    'fi':'100%',
    'ir':'100%',
    'shifts':'100%',
    'dispatch':'100%',
    'arrests':'100%',
    'ic':'100%',
    'demarrests':'100%',
    'ocnd':'100%'
    }

catval = {
    '1d':'5%',
    '1w':'25%',
    '1m':'50%',
    '1y':'75%',
    '5y':'100%',
    'all':'100%',
    }

In [28]:
impath = 'X.png'
rectheight = 16
rectwidth = 16

#### Row Labels

In [29]:
im = Image.open(impath)
w = im.width
h = im.height
x = 50
canvas = Image.new('RGB',((w+x),(h)),'hsl(0,0%,100%)')
canvas.paste(im,(x,0))
im = None

In [30]:
draw = ImageDraw.Draw(canvas)
for i in officer_table.index:
    officer_id = officer_table.officer_id.loc[i]
    text = str(officer_id)
    font = ImageFont.truetype('VeraMono.ttf', rectheight)
    fontWidth, fontHeight = font.getsize(text)
    
    xpos = 5
    ypos = i * rectheight
    draw.text((xpos,ypos),text=text,font=font,fill="black")
    
    linex = w+x
    liney = i * rectheight
    draw.line([(0,liney),(linex,liney)],width=1,fill='black')

In [31]:
#canvas.save('/Users/damoncrockett/Desktop/tmp.png')

#### Column Labels

In [32]:
w = canvas.width
h = canvas.height
y = 500
unicanvas = Image.new('RGB',((w),(h+y)),'hsl(0,0%,100%)')
unicanvas.paste(canvas,(0,y))
canvas = None

In [33]:
draw = ImageDraw.Draw(unicanvas)
for i in limefeat_alphasort.index:
    feature = limefeat_alphasort.featstr.loc[i]
    text = str(feature)
    
    font = ImageFont.truetype('VeraMono.ttf', rectwidth)
    fontWidth, fontHeight = font.getsize(text)
    
    imtmp = Image.new('RGBA', (fontWidth, fontHeight))
    drawtmp = ImageDraw.Draw(imtmp)
    drawtmp.text((0,0),text=text,font=font,fill="black")
    imtmp = imtmp.rotate(90,expand=1)
    
    xpos = i * rectwidth + x - 3
    ypos = y - imtmp.height - 2
    unicanvas.paste(imtmp,(xpos,ypos),imtmp)
    
    linex = i * rectwidth + x
    liney = y
    draw.line([(linex,liney),(linex,unicanvas.height)],width=1,fill='black')

In [34]:
unicanvas.save('/Users/damoncrockett/Desktop/tmp.png')